Программа для размеки документов построчно.

In [8]:
from tkinter import *
from tkinter import filedialog

import cv2
import numpy as np
from PIL import Image, ImageTk
import pytesseract

import os

In [9]:
PATH = r"C:/Users/danii/Dropbox/Работа/04.Данные/Учебник/"
WIDTH = 500
HEIGHT =700

OFFSET_ROW = 2
BOLD_ROW = 1
REGULAR_ROW = 0

COLOR_OFFSET_ROW = (0, 0, 255) 
COLOR_BOLD_ROW = (255, 0, 0)
COLOR_REGULAR_ROW = (0, 255, 0)

files = []
for end in ('.jpg', '.png', '.jpeg'):
    files += [each for each in os.listdir(PATH) if each.endswith(end)]

In [10]:
def read_img(name_file):
    with open(name_file, "rb") as f:
        chunk = f.read()
    chunk_arr = np.frombuffer(chunk, dtype=np.uint8)
    img = cv2.imdecode(chunk_arr, cv2.IMREAD_COLOR)
    return img

In [11]:
class App:
    def __init__(self):
        self.root = Tk()

        # создаем рабочую область
        self.frame = Frame(self.root)
        self.frame.grid()
        
        self.cv_img = None
        self.image = None
        self.photo = None
        self.box = []
        self.name_box = ""
        self.index_file = 0
        # кнопка открытия файла
        self.btn_open_img = Button(self.frame, text="Открыть картинку", 
                                  command=self.my_event_handler)
        
        self.btn_open_img.grid(row=1, column=1)
        
       
        # кнопка следующая картинка 
        self.btn_next = Button(self.frame, text="<",
                                  command=self.my_back)
        self.btn_next.grid(row=2, column=1)
        self.btn_next = Button(self.frame, text=">",
                                  command=self.my_next)
        self.btn_next.grid(row=2, column=2)
         # кнопка сохранения массива
        self.btn_save_box = Button(self.frame, text="Сохранить данные",
                                  command=self.save_box)
        self.btn_save_box.grid(row=3, column=2)
         
        # Добавим изображение
        self.canvas = Canvas(self.root, height=HEIGHT, width=WIDTH)
        self.canvas.bind("<ButtonPress-1>", self.click)
        self.canvas.grid(row=2, column=1)
        
        self.my_event_handler(file_name=files[self.index_file])
        
        self.root.mainloop()
        
        
    def my_next(self):
        self.save_box()
        self.index_file += 1
        if self.index_file == len(files):
            self.index_file = 0
        self.my_event_handler(files[self.index_file])
        
    def my_back(self):
        self.save_box()
        self.index_file -= 1
        if self.index_file == -1:
            self.index_file = len(files)-1
        self.my_event_handler(files[self.index_file]) 
        
    def my_event_handler(self, file_name=None):
        if file_name is None:
            filename = filedialog.askopenfilename(initialdir=PATH, 
                                          title="Select file", 
                                          filetypes=(("img", "*.jpg *.png *.jpeg"),
                                                  ("all files", "*.*")))  
        else:
            filename = PATH + r'/'+ file_name
        print(filename)
        self.dir = os.path.dirname(filename)
        self.name = os.path.basename(filename)
        self.name_box = os.path.join(self.dir, self.name+".npy")
        self.box = []
        self.cv_img = read_img(filename)
        
        if os.path.exists(self.name_box):
            exist_info_box = True
            with open(self.name_box, 'rb') as f:
                self.box = np.load(f).tolist() 
        else:
            gray = cv2.cvtColor(self.cv_img, cv2.COLOR_BGR2GRAY)
            box = pytesseract.image_to_data(gray, output_type=pytesseract.Output.DICT)
            n_boxes = len(box["text"])
            for i in range(n_boxes):
                x0, y0, h0, w0 = int(box['left'][i]), int(box['top'][i]), int(box['height'][i]), int(box['width'][i])
                if (box["level"][i]==4) and (h0 > 5) and (w0 > 5):
                    self.box.append([REGULAR_ROW, x0, y0, h0, w0])
        
        self.draw_img()
        
    def draw_img(self):
        h = self.cv_img.shape[0]
        w = self.cv_img.shape[1]
        
        img_cope = self.cv_img.copy()
        for info_word in self.box:
            font, x0, y0, h0, w0 = info_word
            
            color = (155, 155, 155)
            if font == REGULAR_ROW:
                color = COLOR_REGULAR_ROW
            elif font == BOLD_ROW:
                color = COLOR_BOLD_ROW
            elif font == OFFSET_ROW:
                color = COLOR_OFFSET_ROW
            border = 1
            
            cv2.rectangle(img_cope, (x0, y0), (x0+w0, y0+h0), color, border)
        coef = h/w
        img = cv2.resize(img_cope, (WIDTH, round(coef*WIDTH)))
        
        self.image = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        self.photo = ImageTk.PhotoImage(image = self.image)
        self.c_image = self.canvas.create_image(0, 0, anchor='nw', image=self.photo)
        self.canvas.grid(row=2, column=1)

    def click(self, e):
        w = self.cv_img.shape[1]
        coef = w/WIDTH
        for i in range(len(self.box)):
            font, x0, y0, h0, w0 = self.box[i]
            x1 = e.x*coef
            y1 = e.y*coef
            if (x0 < x1) and (x1 < x0+w0) and (y0 < y1) and (y1 < y0+h0):
                if font == REGULAR_ROW:
                    self.box[i][0] = BOLD_ROW
                elif font == BOLD_ROW:
                    self.box[i][0] = OFFSET_ROW
                elif font == OFFSET_ROW:
                    self.box[i][0] = REGULAR_ROW
        self.draw_img()
        
    def save_box(self):
        N = len(self.box)
        M = 5 # x, y, type
        array = np.zeros((N, M), np.uint16)
        for i in range(N):
            font, x0, y0, h0, w0 = self.box[i]
            array[i][0] = font
            array[i][1] = x0
            array[i][2] = y0
            array[i][3] = h0
            array[i][4] = w0
        with open(self.name_box, 'wb') as f:
            np.save(f, array)
        
app= App()

C:/Users/danii/Dropbox/Работа/04.Данные/Учебник//Messia_t1_1978ru - 16.png
C:/Users/danii/Dropbox/Работа/04.Данные/Учебник//Messia_t1_1978ru - 22.png
C:/Users/danii/Dropbox/Работа/04.Данные/Учебник//Messia_t1_1978ru - 28.png
C:/Users/danii/Dropbox/Работа/04.Данные/Учебник//Messia_t1_1978ru - 3.png
C:/Users/danii/Dropbox/Работа/04.Данные/Учебник//Messia_t1_1978ru - 32.png
C:/Users/danii/Dropbox/Работа/04.Данные/Учебник//Messia_t1_1978ru - 37.png
C:/Users/danii/Dropbox/Работа/04.Данные/Учебник//Messia_t1_1978ru - 4.png
C:/Users/danii/Dropbox/Работа/04.Данные/Учебник//Messia_t1_1978ru - 41.png
C:/Users/danii/Dropbox/Работа/04.Данные/Учебник//Messia_t1_1978ru - 49.png
C:/Users/danii/Dropbox/Работа/04.Данные/Учебник//Messia_t1_1978ru - 5.png
C:/Users/danii/Dropbox/Работа/04.Данные/Учебник//Messia_t1_1978ru - 54.png
C:/Users/danii/Dropbox/Работа/04.Данные/Учебник//Messia_t1_1978ru - 6.png
C:/Users/danii/Dropbox/Работа/04.Данные/Учебник//Messia_t1_1978ru - 7.png
C:/Users/danii/Dropbox/Работа/

In [40]:
os.path.join(PATH, files[0])

'C:\\Users\\danii\\Dropbox\\Работа\\04.Данные\\Разное\\аннотация1.png'